Obtaining the EfficientNetB0 Model form tensorflow and adding extra layers

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

IMAGE_SIZE = 224

imageFolder = "Sub Model Dataset Path"
CLASSES = os.listdir(imageFolder)
num_classes = len(CLASSES)

print(CLASSES)
print(num_classes)

base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False)

base_model.trainable = True

model = tf.keras.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

adam_opt = Adam(learning_rate = 0.0001)
model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])

Creating an Image Data Generator function to obtain the images from local directories

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_folder = "Train Folder Path"
test_folder = "Test Folder Path"

train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale = 1./255
)

train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size=16,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = True
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size = (IMAGE_SIZE, IMAGE_SIZE),
    batch_size=16,
    class_mode = 'categorical',
    color_mode = 'rgb',
    shuffle = False
)

Calculates and creates the class weights for each classes that balances how often a class gets trained to the model

In [4]:
from sklearn.utils.class_weight import compute_class_weight
y_train = train_generator.classes
classes = np.array(train_generator.classes)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = {classes[i]: class_weights[i] for i in range(len(classes))}

Defines callbacks that will monitor the training process and makes adjustment if needed

In [ ]:
EPOCHS = 30
best_model_file = "Directory path /EfficientNetB0-Sub_Model.keras"

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

callbacks = [
    ModelCheckpoint(best_model_file, verbose=1, save_best_only=True, monitor="val_accuracy"),
    ReduceLROnPlateau(monitor="val_loss", patience=2, factor=0.5, verbose=1, min_lr=1e-6),
    EarlyStopping(monitor="val_accuracy", patience=12,verbose=1)
]

Training / Fitting of the EfficientNet Model

In [ ]:
result = model.fit(
    train_generator, epochs=EPOCHS, validation_data=test_generator, callbacks=callbacks, class_weight=class_weight_dict
)

Displays the best / highest accuracy obtained during training

In [ ]:
best_val_acc_epoch = np.argmax(result.history['val_accuracy'])
best_val_acc = result.history['val_accuracy'][best_val_acc_epoch]
print("Best validation accuracy : " + str(best_val_acc))


Displays the Accuracy and Loss graph during each epoch

In [ ]:
# Accuracy Plot
plt.plot(result.history['accuracy'], label='train acc')
plt.plot(result.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()

# Loss Plot
plt.plot(result.history['loss'], label='train loss')
plt.plot(result.history['val_loss'], label='val loss')
plt.legend()
plt.show()

Saving the model to local device

In [9]:
MODEL_NAME = "EfficientNetB0-Sub_Model.keras"
model.save(MODEL_NAME)

Converting the model into tensorflow lite for android use

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('EfficientNetB0-Sub_Model.tflite', 'wb') as f:
    f.write(tflite_model)